B_76 
Shivam Gupta
Practical 3

In [3]:
import datetime
import pymysql

In [4]:
host1 = "bpfz8vv06ufsw48c4xga-mysql.services.clever-cloud.com"
user1 = "uz9woou9dyaujasp"
password1 = "gBR1RNo9i0gQUFwpwNGL"
database1="bpfz8vv06ufsw48c4xga"

conn1 = pymysql.connect(host=host1,
                             user=user1,
                             password=password1,
                             database=database1,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cur = conn1.cursor()





conn2 = pymysql.connect(host=host2,
                             user=user2,
                             password=password2,
                             database=database2,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cur2 = conn2.cursor()

In [5]:

creditCard= input("Enter your credit card number: ")
pin= int(input("Enter your pin: "))
amount= int(input("Enter amount: "))


cur.execute("Select * from creditcards where cardno=%s and pin=%s",(creditCard,pin))
if cur.rowcount==0:
    print("Invalid credit card number or pin")
else:
    print("Valid credit card number and pin.")
conn1.commit()

cur.execute("select limit_ from creditcards where cardno=%s and pin=%s",(creditCard,pin))
if amount>cur.fetchone()['limit_']:
    print("Limit Exceded!!!")
else:
    print("Processing transaction\n")

cur.execute("select mappedaccno from creditcards where cardno=%s and pin=%s",(creditCard,pin))
accno=cur.fetchone()['mappedaccno']


host2 = "bplnbwn14splzds4knfx-mysql.services.clever-cloud.com"
user2 = "umugihupkobaisc7"
password2 = "DtyvWSt23HWmV4ssG4yb"
database2="bplnbwn14splzds4knfx"


cur2.execute("select balance from scbank where accno=%s",(accno))
if cur2.rowcount==0:
    cur2.execute("select balance from citibank where accno=%s",(accno))
    if cur2.rowcount==0:
        print("Invalid account number")
    else:
        balance=cur2.fetchone()['balance']
        if amount>balance:
            print("Insufficient balance")
        else:
            cur2.execute("update citibank set balance=balance-%s where accno=%s",(amount,accno))
            conn2.commit()
            balance=balance-amount
            print("Transaction successful CitiBank")
            print("Card number: ",creditCard)
            print("Amount: ",amount)
            print("Balance: ",balance)
            print("Account number: ",accno)
            print("Transaction type: personal")
            print("Transaction status: successful\n\n")

else:
    balance=cur2.fetchone()['balance']
    if amount>balance:
        print("Insufficient balance")
    else:
        cur2.execute("update scbank set balance=balance-%s where accno=%s",(amount,accno))
        conn2.commit()
        balance=balance-amount
        print("Transaction successful SC_Bank")
        print("Card number: ",creditCard)
        print("Amount: ",amount)
        print("Balance: ",balance)
        print("Account number: ",accno)
        print("Transaction type: personal")
        print("Transaction status: successful")

amount=str(amount)
dateTime=datetime.datetime.now()
cur2.execute("insert into cardtranslog (transdt,accno,amount) values(%s,%s,%s)",(dateTime,accno,amount))
cur.execute("insert into paymentlog (cardno,transdt,purpose,paymentamount) values(%s,%s,%s,%s)",(creditCard,dateTime,"personal",amount))
conn2.commit()
conn1.commit()
conn1.close()
conn2.close()



Enter your credit card number: 11
Enter your pin: 1234
Enter amount: 200
Valid credit card number and pin.
Processing transaction

Transaction successful SC_Bank
Card number:  11
Amount:  200
Balance:  800.0
Account number:  101
Transaction type: personal
Transaction status: successful
